# Etape 2.2 : Fusion et enrichissement

## Charger les consommations nettoyées (depuis Parquet)

In [ ]:
from pyspark.sql import SparkSession
import pandas as pd

# Pandas ne peut pas lire le parquet seul : lecture du Parquet avec Spark, puis convertir en Pandas
spark = SparkSession.builder \
    .appName("ECF2 - Fusion Enrichissement") \
    .master("spark://spark-master:7077") \
    .getOrCreate()

df_conso_spark = spark.read.parquet("../output/consommations_clean")
df_conso = df_conso_spark.toPandas()

df_conso.shape, df_conso.head()

# Chemin météo nettoyée
METEO_PATH = "../output/meteo_clean.csv"

df_meteo = pd.read_csv(METEO_PATH)
# Chargement des tarifs 
TARIFS_PATH = "../data_ecf/tarifs_energie.csv"

df_tarifs = pd.read_csv(TARIFS_PATH)
# Chargement des bâtiments 
df_bat = pd.read_csv("../data_ecf/batiments.csv")
df_bat

,batiment_id,nom,type,commune,surface_m2,annee_construction,classe_energetique,nb_occupants_moyen
0,BAT0001,Ecole Paris 1,ecole,Paris,1926,1978,E,225
1,BAT0002,Ecole Paris 2,ecole,Paris,1156,2004,C,402
2,BAT0003,Ecole Paris 3,ecole,Paris,1695,2014,D,219
3,BAT0004,Mediatheque Paris 4,mediatheque,Paris,907,2019,C,121
4,BAT0005,Piscine Paris 5,piscine,Paris,3913,1950,G,242
...,...,...,...,...,...,...,...,...
141,BAT0142,Piscine Toulon 142,piscine,Toulon,3774,2010,D,223
142,BAT0143,Piscine Toulon 143,piscine,Toulon,3335,1961,F,108
143,BAT0144,Piscine Toulon 144,piscine,Toulon,2496,1997,E,256
144,BAT0145,Mediatheque Toulon 145,mediatheque,Toulon,1204,2020,C,84


## Fusionner avec les données meteo (sur commune et timestamp arrondi à l'heure) et bâtiments

In [12]:
# # ajouter la commune aux consommations
# df = df_conso.merge(df_bat, on="batiment_id", how="left", validate="m:1")
# print("Shape après merge bâtiments:", df.shape)
# df
# # 1) Sécuriser les types datetime
# df_conso["timestamp"] = pd.to_datetime(df_conso["timestamp"], errors="coerce")
# df_meteo["timestamp"] = pd.to_datetime(df_meteo["timestamp"], errors="coerce")

# # df_conso

# df["commune"] = df["commune"].astype(str).str.strip()
# df_meteo["commune"] = df_meteo["commune"].astype(str).str.strip()

# # # 2) Créer la clé horaire des deux côtés
# df_conso["ts_hour"] = df_conso["timestamp"].dt.floor("H")
# df_meteo["ts_hour"] = df_meteo["timestamp"].dt.floor("H")

# # 3) Fusion météo (commune + heure)
# df_fusion = df.merge(
#     df_meteo.drop(columns=["timestamp"]),   # on garde ts_hour
#     on=["commune", "ts_hour"],
#     how="left",
#     validate="m:1"
# )

# 1) Merge conso + bâtiments (ajoute commune)
df = df_conso.merge(df_bat, on="batiment_id", how="left", validate="m:1")
print("Shape après merge bâtiments:", df.shape)
print("Commune manquante:", df["commune"].isna().sum())

# 2) Sécuriser datetime (sur df + météo)
df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
df_meteo["timestamp"] = pd.to_datetime(df_meteo["timestamp"], errors="coerce")

# 3) Nettoyer commune (espaces)
df["commune"] = df["commune"].astype(str).str.strip()
df_meteo["commune"] = df_meteo["commune"].astype(str).str.strip()

# 4) Créer la clé horaire (SUR df, pas df_conso)
df["ts_hour"] = df["timestamp"].dt.floor("H")
df_meteo["ts_hour"] = df_meteo["timestamp"].dt.floor("H")

# 5) Fusion météo
df_fusion = df.merge(
    df_meteo.drop(columns=["timestamp"]),
    on=["commune", "ts_hour"],
    how="left",
    validate="m:1"
)

print("Shape après fusion météo:", df_fusion.shape)
print("Lignes sans météo:", df_fusion["temperature_c"].isna().sum())
print("Taux sans météo (%):", round(df_fusion["temperature_c"].isna().mean()*100, 2))

print("df_fusion créé:", "df_fusion" in globals(), "shape:", df_fusion.shape if "df_fusion" in globals() else None)


Shape après merge bâtiments: (100000, 13)
Commune manquante: 0
Shape après fusion météo: (100000, 23)
Lignes sans météo: 75663
Taux sans météo (%): 75.66
df_fusion créé: True shape: (100000, 23)


## Fusionner avec les tarifs pour calculer le coût financier

In [13]:
# préparation des dates 
df_tarifs["date_debut"] = pd.to_datetime(df_tarifs["date_debut"])
df_tarifs["date_fin"] = pd.to_datetime(df_tarifs["date_fin"])

# ajout colonne “date_jour” côté consommations
df_fusion["date_jour"] = pd.to_datetime(df_fusion["timestamp"]).dt.floor("D")

# Jointure sur type_energie + filtragdf_tmp = df_fusion.merge(df_tarifs, on="type_energie", how="left")
df_tmp = df_fusion.merge(df_tarifs, on="type_energie", how="left")
df_tmp = df_tmp[
    (df_tmp["date_jour"] >= df_tmp["date_debut"]) &
    (df_tmp["date_jour"] <= df_tmp["date_fin"])
].copy()


# Sécurité anti “multi-match” (au cas où une date tombe dans 2 périodes par erreur)

df_tmp = df_tmp.sort_values(["batiment_id", "timestamp", "type_energie", "date_debut"]) \
               .drop_duplicates(["batiment_id", "timestamp", "type_energie"], keep="last")

# clalcul du coût
df_tmp["cout_eur"] = df_tmp["conso_clean"] * df_tmp["tarif_unitaire"]

# audit 
print("Tarif manquant :", df_tmp["tarif_unitaire"].isna().sum())
print("Coût manquant  :", df_tmp["cout_eur"].isna().sum())
df_tmp[["type_energie","date_jour","tarif_unitaire","conso_clean","cout_eur"]].head()

print("df_tmp créé:", "df_tmp" in globals(), "shape:", df_tmp.shape if "df_tmp" in globals() else None)


Tarif manquant : 0
Coût manquant  : 0
df_tmp créé: True shape: (100000, 28)


## Créer des features dérivées

In [22]:
df_enrichi = df_tmp
# 1 : Consommation par occupant : normalisation de la consommation par le nombre d’occupants.

df_enrichi["conso_par_occupant"] = (
    df_enrichi["conso_clean"] / df_enrichi["nb_occupants_moyen"]
)

# éviter inf / NaN si nb_occupants = 0
df_enrichi.loc[df_enrichi["nb_occupants_moyen"] <= 0, "conso_par_occupant"] = None

# 2 : Consommation par m2

df_enrichi["conso_par_m2"] = (
    df_enrichi["conso_clean"] / df_enrichi["surface_m2"]
)

df_enrichi.loc[df_enrichi["surface_m2"] <= 0, "conso_par_m2"] = None


# 3 : Cout journalier, mensuel, annuel

# securisation de la date 
df_enrichi["date_jour"] = pd.to_datetime(df_enrichi["date_jour"])

# couût journalier par bât
cout_journalier = (
    df_enrichi
    .groupby(["batiment_id", "date_jour"], as_index=False)
    .agg(cout_journalier_eur=("cout_eur", "sum"))
)

# coût mensuel

df_enrichi["mois"] = df_enrichi["date_jour"].dt.to_period("M")

cout_mensuel = (
    df_enrichi
    .groupby(["batiment_id", "mois"], as_index=False)
    .agg(cout_mensuel_eur=("cout_eur", "sum"))
)

# couût annuel 
df_enrichi["annee"] = df_enrichi["date_jour"].dt.year

cout_annuel = (
    df_enrichi
    .groupby(["batiment_id", "annee"], as_index=False)
    .agg(cout_annuel_eur=("cout_eur", "sum"))
)

# 4 : Indice de performance energetique (IPE)

conso_annuelle = (
    df_enrichi
    .groupby(["batiment_id", "annee"], as_index=False)
    .agg(conso_annuelle_kwh=("conso_clean", "sum"))
)

conso_annuelle = conso_annuelle.merge(
    df_enrichi[["batiment_id", "surface_m2"]].drop_duplicates(),
    on="batiment_id",
    how="left"
)

conso_annuelle["IPE_kwh_m2_an"] = (
    conso_annuelle["conso_annuelle_kwh"] / conso_annuelle["surface_m2"]
)

df_enrichi = df_enrichi.merge(
    conso_annuelle[["batiment_id", "annee", "IPE_kwh_m2_an"]],
    on=["batiment_id", "annee"],
    how="left"
)

# moyenne par type de bât 
moyenne_type = (
    df_enrichi
    .groupby("type")["conso_par_m2"]
    .mean()
    .rename("moyenne_conso_type")
)

#ecart 
df_enrichi = df_enrichi.join(moyenne_type, on="type")

df_enrichi["ecart_moyenne_categorie"] = (
    df_enrichi["conso_par_m2"] - df_enrichi["moyenne_conso_type"]
)

df_enrichi.dtypes

batiment_id                        object
timestamp                  datetime64[ns]
conso_clean                       float64
unite                              object
date                               object
type_energie                       object
nom                                object
type                               object
commune                            object
surface_m2                          int64
annee_construction                  int64
classe_energetique                 object
nb_occupants_moyen                  int64
ts_hour                    datetime64[ns]
temperature_c                     float64
humidite_pct                      float64
rayonnement_solaire_wm2           float64
vitesse_vent_kmh                  float64
precipitation_mm                  float64
jour                              float64
mois                            period[M]
jour_semaine                      float64
saison                             object
date_jour                  datetim

## Dataset final `output/consommations_enrichies.csv` et `.parquet`


In [23]:
import os

OUTPUT_DIR = "../output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# export csv 
CSV_PATH = os.path.join(OUTPUT_DIR, "consommations_enrichies.csv")

df_enrichi.to_csv(
    CSV_PATH,
    index=False,
    encoding="utf-8"
)

print("CSV exporté :", CSV_PATH)

# export parquet 
PARQUET_PATH = os.path.join(OUTPUT_DIR, "consommations_enrichies.parquet")

df_enrichi.to_parquet(
    PARQUET_PATH,
    index=False
)

print("Parquet exporté :", PARQUET_PATH)


CSV exporté : ../output/consommations_enrichies.csv
Parquet exporté : ../output/consommations_enrichies.parquet


## Dictionnaire de donnees (description de toutes les colonnes)

In [19]:
data_dictionary = [
    # Identité
    ("batiment_id", "Identifiant unique du bâtiment", "clé", "Référentiel bâtiments"),
    ("nom", "Nom du bâtiment", "texte", "Référentiel bâtiments"),
    ("type", "Type de bâtiment (école, bureau, hôpital…)", "catégoriel", "Référentiel bâtiments"),
    ("commune", "Commune du bâtiment", "texte", "Référentiel bâtiments"),

    # Temps
    ("timestamp", "Horodatage de la mesure", "datetime", "Consommations"),
    ("date_jour", "Date (jour)", "date", "Dérivé timestamp"),
    ("mois", "Mois de la mesure", "période", "Dérivé timestamp"),
    ("annee", "Année de la mesure", "entier", "Dérivé timestamp"),

    # Consommation
    ("conso_clean", "Consommation énergétique nettoyée", "kWh", "Consommations nettoyées"),
    ("type_energie", "Type d’énergie (élec, gaz…)", "catégoriel", "Consommations"),
    ("unite", "Unité de mesure", "texte", "Consommations"),

    # Bâtiment
    ("surface_m2", "Surface du bâtiment", "m²", "Référentiel bâtiments"),
    ("nb_occupants_moyen", "Nombre moyen d’occupants", "personnes", "Référentiel bâtiments"),
    ("classe_energetique", "Classe énergétique du bâtiment", "A–G", "Référentiel bâtiments"),

    # Météo
    ("temperature_c", "Température extérieure", "°C", "Météo"),
    ("humidite_pct", "Humidité relative", "%", "Météo"),
    ("rayonnement_solaire_wm2", "Rayonnement solaire", "W/m²", "Météo"),
    ("vitesse_vent_kmh", "Vitesse du vent", "km/h", "Météo"),
    ("precipitation_mm", "Précipitations", "mm", "Météo"),

    # Tarifs & coûts
    ("tarif_unitaire", "Tarif énergétique unitaire", "€/kWh", "Tarifs"),
    ("cout_eur", "Coût énergétique de la mesure", "€", "Calculé"),

    # Features dérivées
    ("conso_par_occupant", "Consommation par occupant", "kWh/personne", "Calculé"),
    ("conso_par_m2", "Consommation par m²", "kWh/m²", "Calculé"),
    ("IPE_kwh_m2_an", "Indice de performance énergétique", "kWh/m²/an", "Calculé"),
    ("ecart_moyenne_categorie", "Écart à la moyenne du type de bâtiment", "kWh/m²", "Calculé"),
]

#mise en df
dict_df = pd.DataFrame(
    data_dictionary,
    columns=["colonne", "description", "unité/type", "origine"]
)

dict_df

# export du dictionnaire 
DICT_PATH = os.path.join(OUTPUT_DIR, "dictionnaire_donnees.csv")
dict_df.to_csv(DICT_PATH, index=False, encoding="utf-8")

print("Dictionnaire exporté :", DICT_PATH)


Dictionnaire exporté : ../output/dictionnaire_donnees.csv
